In [1]:
import numpy as np

In [2]:
from random import random
from scipy.optimize import minimize

from qiskit import *
from qiskit.circuit.library.standard_gates import U2Gate
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.algorithms import NumPyEigensolver

In [3]:
def hamiltonian_operator(a, b, c, d):
    pauli_dict = {
        'paulis': [{"coeff": {"imag": 0.0, "real": a}, "label": "I"},
                   {"coeff": {"imag": 0.0, "real": b}, "label": "Z"},
                   {"coeff": {"imag": 0.0, "real": c}, "label": "X"},
                   {"coeff": {"imag": 0.0, "real": d}, "label": "Y"}
            
        ]
    }
    
    return WeightedPauliOperator.from_dict(pauli_dict)

In [4]:
scale = 10
a, b, c, d = (scale*random(), scale*random(),
             scale*random(), scale*random())
H = hamiltonian_operator(a, b, c, d)

In [5]:
exact_result = NumPyEigensolver(H).run()
reference_energy = min(np.real(exact_result.eigenvalues))
print("The exact ground state energy is: {}", format(reference_energy))

The exact ground state energy is: {} 3.4638824438234415


In [6]:
def quantum_state_prepartion(circuit, parameters):
    q = circuit.qregs[0]
    circuit.rx(parameters[0], q[0])
    circuit.ry(parameters[1], q[0])
    return circuit

In [7]:
H_gate = U2Gate(0, np.pi).to_matrix()
print("H_gate: ")
print((H_gate*np.sqrt(2)).round(5))

Y_gate = U2Gate(0, np.pi/2).to_matrix()
print("Y_gate:")
print((Y_gate*np.sqrt(2)).round(5))

H_gate: 
[[ 1.+0.j  1.-0.j]
 [ 1.+0.j -1.+0.j]]
Y_gate:
[[ 1.+0.j -0.-1.j]
 [ 1.+0.j  0.+1.j]]


In [8]:
def vqe_circuit(parameters, measure):
    q = QuantumRegister(1)
    c = ClassicalRegister(1)
    circuit = QuantumCircuit(q, c)
    
    circuit = quantum_state_prepartion(circuit, parameters)
    
    # measurement
    if measure == 'Z':
        circuit.measure(q[0], c[0])
    elif measure == 'X':
        circuit.u2(0, np.pi, q[0])
        circuit.measure(q[0], c[0])
    elif measure == 'Y':
        circuit.u2(0, np.pi/2, q[0])
        circuit.measure(q[0], c[0])
    else:
        raise ValueError('Not valid input for measurement: input should be "X" or "Y" or "Z"')
        
    return circuit

In [9]:
def quantum_module(parameters, measure):
    if measure == 'I':
        return 1
    elif measure == 'Z':
        circuit = vqe_circuit(parameters, 'Z')
    elif measure == 'X':
        circuit = vqe_circuit(parameters, 'X')
    elif measure == 'Y':
        circuit = vqe_circuit(parameters, 'Y')
    else:
        raise ValueError('Not valid input for measurement: input should be "I" or "X" or "Y"')
        
    shots = 8192
    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts()
    
    expectation_value = 0
    for measure_result in counts:
        sign = +1
        if measure_result == '1':
            sign = -1
        expectation_value += sign*counts[measure_result]/shots
        
    return expectation_value

In [10]:
def pauli_operator_to_dict(pauli_operator):
    d = pauli_operator.to_dict()
    paulis = d['paulis']
    paulis_dict = {}
    
    for x in paulis:
        label = x['label']
        coeff = x['coeff']['real']
        paulis_dict[label] = coeff
        
    return paulis_dict
pauli_dict = pauli_operator_to_dict(H)

In [11]:
def vqe(parameters):
    
    quantum_module_I = pauli_dict['I']*quantum_module(parameters, 'I')
    quantum_module_Z = pauli_dict['Z']*quantum_module(parameters, 'Z')
    quantum_module_X = pauli_dict['X']*quantum_module(parameters, 'X')
    quantum_module_Y = pauli_dict['Y']*quantum_module(parameters, 'Y')
    
    classical_adder = quantum_module_I + quantum_module_Z + quantum_module_X + quantum_module_Y
    
    return classical_adder

In [17]:
parameters_array = np.array([np.pi, np.pi])
tol = 1e-3

vqe_result = minimize(vqe, parameters_array, method="Powell", tol=tol)
print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy from VQE algorithm is: {}'.format(vqe_result.fun))

/home/sushant/Documents/mysoft/anaconda3/envs/vqiskit/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The QuantumCircuit.u2 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You can use the general 1-qubit gate QuantumCircuit.u instead: u2(φ,λ) = u(π/2, φ, λ). Alternatively, you can decompose it interms of QuantumCircuit.p and QuantumCircuit.sx: u2(φ,λ) = p(π/2+φ) sx p(π/2+λ) (1 pulse on hardware).
  if sys.path[0] == '':


The exact ground state energy is: 3.4638824438234415
The estimated ground state energy from VQE algorithm is: 3.470420064683463
